# <center> PROCESAMIENTO DIGITAL DE SEÑALES DE AUDIO</center>
## <center> Banco de filtros auditivos</center>      

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import wavfile

import IPython.display as ipd

**NOTA:** *Las siguientes dos celdas solo son necesarias para descargar el archivo de ejemplo. Ignórelas si va a trabajar con sus propios archivos de audio.*

In [ ]:
!pip install wget

In [ ]:
import wget

**NOTA:** *Las siguientes celdas instalan e importan la biblioteca [librosa](https://librosa.org/) para procesamiento de audio y música en python.*

In [ ]:
!pip install librosa

In [ ]:
import librosa
import librosa.display

In [ ]:
librosa.__version__

### Descripción

Este ejercicio busca estudiar bancos de filtros usados simular la **selectividad en frecuencia del sistema auditivo**, en particular su **distribución no lineal** y su **ancho de banda variable**. Este tipo de banco de filtros se utilizan frecuentemente como una primera etapa en tareas de procesamiento de audio y permiten construir una representación adecuada de la señal de audio.

La tarea planteada consiste en estudiar el **banco de filtros en escala mel** implementado en la biblioteca [librosa](https://librosa.org/), analizando sus parámetros y su función en el diseño. Luego se aplica el banco de filtros diseñado para filtrar una señal de audio y se analiza el efecto del valor de los parámetros en la representación espectral obtenida.

### Cómo correr el notebook
Se puede bajar y correr el notebook de forma local en una computadora.

O también se puede correr en Google Colab usando el siguiente enlace. 

<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/mrocamora/audio-dsp/blob/main/notebooks/audioDSP-mel_filterbank_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

### Obtener un archivo

La siguiente celda descarga un archivo de audio de ejemplo.

In [ ]:
# download audio file to use
wget.download('https://github.com/mrocamora/audio-dsp/blob/main/audio/superstition.wav?raw=true')

In [ ]:
# read the audio file
filename = 'superstition.wav'
y, sr = librosa.load(filename)

# play audio
ipd.Audio(y, rate=sr)

La siguiente celda grafica la forma de onda.

In [ ]:
# plot audio signal
plt.figure(figsize=(12,8))
ax1 = plt.subplot(2, 1, 1)
librosa.display.waveplot(y, sr=sr)
plt.title('audio waveform')
plt.tight_layout()

### Banco de filtros mel

En lo que sigue se diseña un banco de filtros en escala mel usando [librosa](https://librosa.org/). Estudie los parámetros que recibe la función para el diseño del banco de filtros, analice el resultado obtenido y conteste las siguientes preguntas. Puede resultar útil cambiar la cantidad de filtros del banco.

  1. ¿Cómo es la distribución de la frecuencia central de los filtros del banco?
  2. ¿Qué forma tiene la respuesta en frecuencia de cada filtro?
  3. ¿Cómo varía el ancho de banda de los filtros a medida de que crece la frecuencia?
  4. ¿En qué regiones de frecuencia el banco de filtros tiene más resolución en frecuencia?
  5. ¿Cómo varía la ganancia de los filtros con la frecuencia? ¿A qué tipo de normalización corresponde?

El siguiente código define los parámetros del banco de filtros.

In [ ]:
# number of DFT points
n_fft = 2048

# number of mel-frequency bands
n_mels = 128

# maximum frequency for the analysis
fmax = 4000 

A continuación se construye el banco de filtros y se representa gráficamente su frecuencia central y la magnitud de la respuesta en frecuencia de cada filtro.

In [ ]:
# compute and plot the Mel filter bank
melfb = librosa.filters.mel(sr, n_fft, fmax=fmax, n_mels=n_mels)
freqs = librosa.fft_frequencies(n_fft=n_fft)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
librosa.display.specshow(melfb, x_axis='linear')
plt.xlim([0, fmax])
plt.ylabel('Mel filter')
plt.title('Mel filter bank')
plt.subplot(1, 2, 2)
plt.plot(freqs, melfb.T)
plt.title('Mel filter bank')
plt.xlabel('Frequency [Hz]')
plt.xlim([0, fmax])
plt.tight_layout()

### Cálculo de espectrograma mel
En la siguiente celda se aplica un banco de filtros mel con las mismas características sobre el espectrograma de la señal de audio, para producir un espectrograma mel. Cambie los parámetros del banco de filtros y compare el espectrograma original y el espectrograma mel. 

En particular considere lo siguiente.

  1. ¿Cómo es la resolución en frecuencias del espectrograma original?
  2. ¿En qué rango de frecuencias el espectrograma mel tiene más resolución?

In [ ]:
# 1. Compute spectrogam from STFT
Y = librosa.stft(y, win_length=1024, hop_length=512, n_fft=n_fft, window='hann')
S = np.abs(Y)**2

# 2. apply mel-filterbank to combine FFT bins into Mel-frequency bins
# compute mel-spectrogram
M = librosa.feature.melspectrogram(S=S, n_mels=n_mels, fmax=fmax)

# 3. apply log to convert power to dB
M_log = librosa.power_to_db(M)

In [ ]:
# plot spectrogram and mel-spectrogram
ind_max = np.argmax(freqs > fmax)
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
#librosa.display.specshow(librosa.power_to_db(S[:ind_max, :]), y_coords=freqs[:ind_max], y_axis='linear')
librosa.display.specshow(librosa.power_to_db(S), y_coords=freqs, y_axis='linear')
ax=plt.gca()
ax.set_ylim([0, fmax])
plt.title('spectrogram')
plt.subplot(2, 1, 2)
librosa.display.specshow(M_log, x_axis='time', y_axis='mel', sr=sr, fmax=fmax)
plt.title('mel-spectrogram')
plt.tight_layout()